In [1]:
import pandas as pd
import joblib
import os

# --- Configuration ---
MODEL_PATH = "model/model.pkl" 
FEATURE_FILE_S3 = "s3://ai-trading-copilot-curated/features/fe-yearly-run-2025-10-17-23-39-01/AAPL_features.csv"
TICKER_SYMBOL = 'AAPL'

def infer_local_model(ticker_symbol, feature_file_path, model_path):
    """Loads a local model and predicts on the latest data point from S3."""

    # --- 1. Load Local Model ---
    print(f"Loading model from: {model_path}")
    try:
        model = joblib.load(model_path)
    except FileNotFoundError:
        print(f"ERROR: Model file not found at '{model_path}'.")
        return
    except Exception as e:
        print(f"ERROR: Could not load model: {e}")
        return

    # --- 2. Load and Prepare Latest Data from S3 ---
    print(f"Loading latest features from: {feature_file_path}")
    try:
        # Pandas can read directly from S3 if credentials are configured
        df = pd.read_csv(feature_file_path)
        latest_data = df.tail(1).copy()
    except Exception as e:
        print(f"ERROR: Could not read feature file from S3: {e}")
        return

    # --- 3. Align Features (No changes needed here) ---
    known_non_features = ['date', 'open', 'high', 'low', 'close', 'volume', 'symbol', 'asof', 'target']
    features = [col for col in df.columns if col not in known_non_features]
    
    missing_features = [f for f in features if f not in latest_data.columns]
    if missing_features:
        print(f"ERROR: Missing expected feature columns in data: {missing_features}")
        return

    X_latest = latest_data[features]

    # --- 4. Make Prediction ---
    try:
        prediction = model.predict(X_latest)[0]
        prediction_proba = model.predict_proba(X_latest)[0]
    except Exception as e:
        print(f"ERROR: Prediction failed: {e}")
        return

    # --- 5. Display Result ---
    direction = "Up" if prediction == 1 else "Down"
    confidence = prediction_proba[1] if direction == "Up" else prediction_proba[0]

    print("\n--- Local Inference Complete ---")
    print(f"Prediction for {ticker_symbol} (next day):")
    print(f"  Direction:  {direction}")
    print(f"  Confidence: {confidence:.2%}")

# --- Run the inference ---
if __name__ == '__main__':
    if not os.path.exists(MODEL_PATH):
        print(f"Please download 'model.pkl' and place it at '{MODEL_PATH}'.")
    else:
        infer_local_model(TICKER_SYMBOL, FEATURE_FILE_S3, MODEL_PATH)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


Loading model from: model.pkl


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/pickle.py:1718: UserWarning: [00:11:15] WARNING: /workspace/src/collective/../data/../common/error_msg.h:82: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, th

Loading latest features from: s3://ai-trading-copilot-curated/features/fe-yearly-run-2025-10-17-23-39-01/AAPL_features.csv


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:298: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)



--- ✅ Local Inference Complete ---
Prediction for AAPL (next day):
  Direction:  Down
  Confidence: 56.85%
